# **Parte II: Ejercicios Prácticos**
### *Xavier Tandazo*
___
### **Exercise 1: Caesar Cipher Analysis**
Given the following ciphertext encrypted with a Caesar cipher:

In [5]:
ciphertext = """Al osk lzw twkl gx laewk, al osk lzw ogjkl gx laewk, al osk lzw syw gx oakvge,
al osk lzw syw gx xggdakzfwkk, al osk lzw whguz gx twdawx, al osk lzw whguz gx
afujwvmdalq, al osk lzw kwskgf gx Dayzl, al osk lzw kwskgf gx Vsjcfwkk, al osk
lzw khjafy gx zghw, al osk lzw oaflwj gx vwkhsaj, ow zsv wnwjqlzafy twxgjw mk,
ow zsv fglzafy twxgjw mk, ow owjw sdd ygafy vajwul lg Zwsnwf, ow owjw sdd ygafy
vajwul lzw glzwj osq..."""

In [6]:
def caesar_decrypt(text, shift):
    result = ""
    for ch in text: #aqui recorremos cada caracter
        if ch.isalpha(): #verificamos si es una letra, ya que hay muchos espacios y comas
            base = ord('A') if ch.isupper() else ord('a')
            result += chr((ord(ch) - base - shift) % 26 + base) #transformamos la letra a su indice, le restamos el desplazamiento y volvemos a transformar a letra
        else:
            result += ch #si no es una letra, la dejamos igual
    return result

Ahora vamos a descargar una libreria para validar palabras.

In [ ]:
import nltk
nltk.download('words')

Con esto ya podemos encontrar el mejor score tomando como **guía** el diccionario de palabras y contrastando con cada de las keys que generemos (total 26). 

In [ ]:
from nltk.corpus import words
ENGLISH_VOCAB = set(w.lower() for w in words.words()) #usamos la libreria para tener un vocabulario en ingles

def clean_and_words(s):
    cleaned_chars = []
    for ch in s:
        if ch.isalpha() or ch.isspace():  #mantenemos letras y espacios
            cleaned_chars.append(ch.lower())
        else:
            cleaned_chars.append(' ') #reemplazamos otros caracteres por ' '
    cleaned = ''.join(cleaned_chars)
    return [w for w in cleaned.split() if w] #devolvemos la lista de palabras

def score_with_nltk_vocab(plain):
    tokens = clean_and_words(plain) #obtenemos las palabras
    if not tokens:
        return 0  #evitar division por cero
    valid = sum(1 for w in tokens if w in ENGLISH_VOCAB) #contar palabras validas
    return valid / len(tokens) #proporcion de palabras validas

candidates = []
for k in range(26):
    plain = caesar_decrypt(ciphertext, k) #desciframos con la clave k
    score = score_with_nltk_vocab(plain) #calculamos score basado en vocabulario
    candidates.append((k, score, plain)) #guardamos k, score y texto descifrado

candidates.sort(key=lambda t: t[1], reverse=True)

best = candidates[0] #mejor clave y score
print("Mejor clave con nltk:", best[0], " — score:", best[1])
print("\nTexto descifrado con la mejor clave:\n")
print(best[2]) #imprimimos solo el texto del mejor

print("\nResto de candidatos — score:\n")
for i, (k, score, _) in enumerate(candidates[1:4], 2): #enumeramos desde el segundo mejor
    print(f"{i}) clave {k} — score={score:.3f}")

Mejor clave con nltk: 18  — score: 1.0

Texto descifrado con la mejor clave:

It was the best of times, it was the worst of times, it was the age of wisdom,
it was the age of foolishness, it was the epoch of belief, it was the epoch of
incredulity, it was the season of Light, it was the season of Darkness, it was
the spring of hope, it was the winter of despair, we had everything before us,
we had nothing before us, we were all going direct to Heaven, we were all going
direct the other way...

Resto de candidatos — score:

2) clave 25 — score=0.271
3) clave 0 — score=0.165
4) clave 6 — score=0.153


___
### **Exercise 2: Affine Cipher Implementation**
Given the following ciphertext encrypted with a Caesar cipher:

$$
E(x) = (a \cdot x + b) \mod 26
$$

Donde:

- Plaintext: **CRYPTOGRAPHY**  
- \(a = 5\)  
- \(b = 8\)  
- Índices de letras: \(x = 0 \text{ para A}, 1 \text{ para B}, \dots, 25 \text{ para Z}\)

   **a)** Encrypt the plaintext (show calculations for first 3 letters) 


El proceso a "mano" de las 3 primeras letras sería así, reemplazando todo lo que el enunciado nos proporciona (variables a,b y x) según corresponda con la letra del *Plaintext*:

| Letra | Índice \(x\) | Cálculo \(E(x) = (5x+8) \mod 26\) | Índice cifrado \(y\) | Letra cifrada |
|-------|--------------|----------------------------------|---------------------|---------------|
| **C**     | 2            | (5·2 + 8) mod 26                | 18                  | S             |
| **R**     | 17           | (5·17 + 8) mod 26               | 15                  | P             |
| **Y**     | 24           | (5·24 + 8) mod 26               | 24                  | Y             |

In [14]:
#Ponemos los datos
pla = "CRYPTOGRAPHY" #texto plano
a = 5
b = 8

def let_a_ind(c): #esta sera funcion para convertir letra a indice (A=0,...,Z=25)
    return ord(c.upper()) - ord('A')

def ind_a_let(i): #y esta funcion para convertir indice a letra
    return chr(i + ord('A'))

#Encriptamos
cip = ""
for c in pla:
    x = let_a_ind(c)  #convertimos la letra a indice
    y = (a * x + b) % 26  #formula del enunciado
    cip += ind_a_let(y)  #convertimos de vuelta a letra y agregar al ciphertext

print("Texto cifrado completo:", cip)

Texto cifrado completo: SPYFZAMPIFRY


**b)** Find the decryption key values (multiplicative inverse of a)


Sabemos que la fórmula de descifrado del Affine Cipher:

$$
D(y) = a^{-1} \cdot (y - b) \mod 26
$$

donde:

$$
a \cdot a^{-1} \equiv 1 \ (\text{mod } 26)
$$

Dado que $a = 5$, buscamos $a^{-1}$ tal que:

$$
5 \cdot a^{-1} \equiv 1 \ (\text{mod } 26)
$$

In [ ]:
#Esta funcion srive para encontrar inversa multiplicativa modulo m
def inv_mod(a, m):
    for i in range(1, m): #probamos por todos los posibles valores de 1 a m-1
        if (a * i) % m == 1: #verificamos la condicion de inversa
            return i
    return None #si no existe inversa

a = 5
m = 26
a_inv = inv_mod(a, m) #calcular inversa multiplicativa
print("La inversa es", a_inv)


La inversa es 21


Entonces encontramos que:

$$
a^{-1} = 21
$$

Entonces concluímos que la fórmula de descifrado completa es:

$$
D(y) = 21 \cdot (y - 8) \mod 26
$$

**c)** Verify your encryption by decrypting the first 3 letters 

El proceso a "mano" de las 3 primeras letras cifradas sería así, usando la fórmula de descifrado:

| Letra cifrada | Índice \(y\) | Cálculo \(D(y) = 21 \cdot (y - 8) \mod 26\) | Índice descifrado | Letra descifrada |
|---------------|--------------|--------------------------------------------|-----------------|----------------|
| **S**         | 18           | 21*(18-8) mod 26 = 210 mod 26             | 2               | C              |
| **P**         | 15           | 21*(15-8) mod 26 = 147 mod 26             | 17              | R              |
| **Y**         | 24           | 21*(24-8) mod 26 = 336 mod 26             | 24              | Y              |


In [ ]:
#Ponemos los datos
a_inv = 21
b = 8

#Funciones de conversion letra a indice
def let_a_ind(c):
    return ord(c.upper()) - ord('A')

def ind_a_let(i):
    return chr(i + ord('A'))

#Decframos el plaintext
plain = ""
for c in cip:
    y = let_a_ind(c) #indice de la letra cifrada
    x = (a_inv * (y - b)) % 26 #formula de descifrado
    plain += ind_a_let(x) #convertimos de indice a letra y agregar al plaintext

print("Texto descifrado completo:", plain)


Texto descifrado completo: CRYPTOGRAPHY


**d)** How many valid keys exist for the Affine cipher? Explain your reasoning.

Sabemos que para que una clave `a` sea válida en el Affine Cipher, se debe cumplir que tenga **inversa multiplicativa módulo 26**.  
Es decir, que `a` y 26 deben ser **coprimos**:

$$
\gcd(a, 26) = 1
$$

- Los posibles valores de `a` entre 1 y 25 que cumplen esta condición son:

$$
a = 1, 3, 5, 7, 9, 11, 15, 17, 19, 21, 23, 25
$$

Osea que hay 12 valores posibles para `a`.

- El valor de `b` puede ser cualquier número entre 0 y 25, osea que hay 26 posibilidades.

Por lo tanto, el número total de claves válidas del Affine Cipher es:

$$
12 \cdot 26 = 312
$$

___
#### **Exercise 3: Perfect Secrecy Analysis**
Consider a simple cipher that operates on single bits where:
- Key space: {0, 1}
- Plaintext space: {0, 1}
- Encryption: C = P ⊕ K (XOR operation)
- Each key is chosen with probability 1/2

**a)** Create the complete encryption matrix showing all possible (plaintext, key, ciphertext) combinations

| Plaintext (P) | Key (K) | Ciphertext (C = P ⊕ K) |
|---------------|---------|------------------------|
| 0             | 0       | 0                      |
| 0             | 1       | 1                      |
| 1             | 0       | 1                      |
| 1             | 1       | 0                      |

**b)** Calculate P(C=0) and P(C=1) 

Sabemos que cada clave se elige con probabilidad \(1/2\) y que cada plaintext es igualmente probable.  

De la matriz de cifrado:

| Plaintext (P) | Key (K) | Ciphertext (C = P ⊕ K) |
|---------------|---------|------------------------|
| 0             | 0       | 0                      |
| 0             | 1       | 1                      |
| 1             | 0       | 1                      |
| 1             | 1       | 0                      |

Entonces podemos ver que:

- \(C = 0\) ocurre en dos de las cuatro combinaciones posibles:

$$
P(C=0) = \frac{2}{4} = \frac{1}{2}
$$

- \(C = 1\) ocurre en las otras dos combinaciones:

$$
P(C=1) = \frac{2}{4} = \frac{1}{2}
$$



**c)** Calculate P(P=0|C=0) and P(P=1|C=0) 

Usamos la definición de probabilidad condicional:

$$
P(P=p \mid C=c) = \frac{P(P=p, C=c)}{P(C=c)}
$$

De la matriz de cifrado:

| Plaintext (P) | Key (K) | Ciphertext (C = P ⊕ K) |
|---------------|---------|------------------------|
| 0             | 0       | 0                      |
| 0             | 1       | 1                      |
| 1             | 0       | 1                      |
| 1             | 1       | 0                      |

Para \(C = 0\):

- Combinaciones con \(C=0\): \((P=0,K=0)\) y \((P=1,K=1)\)  
- Probabilidades conjuntas: \(P(P=0, C=0) = 1/4\), \(P(P=1, C=0) = 1/4\)  
- Como \(P(C=0) = 1/2\):

$$
P(P=0 \mid C=0) = \frac{P(P=0, C=0)}{P(C=0)} = \frac{1/4}{1/2} = \frac{1}{2}
$$

$$
P(P=1 \mid C=0) = \frac{P(P=1, C=0)}{P(C=0)} = \frac{1/4}{1/2} = \frac{1}{2}
$$

**d)** Does this cipher achieve perfect secrecy? Prove your answer using Shannon's definition


La definición de Shannon dice que un cifrado logra **secreto perfecto** si la distribución de probabilidad del plaintext es **independiente del ciphertext**, es decir:

$$
P(P=p \mid C=c) = P(P=p) \quad \forall p \in P, c \in C
$$

De nuestro ejercicio:

- Sabemos que 

$$
(P(P=0) = P(P=1) = 1/2)  
$$

- Y del literal c), calculamos:

$$
P(P=0 \mid C=0) = \frac{1}{2} = P(P=0)
$$

$$
P(P=1 \mid C=0) = \frac{1}{2} = P(P=1)
$$

De forma simétrica hacemos que para \(C=1\):

$$
P(P=0 \mid C=1) = \frac{1}{2} = P(P=0)
$$

$$
P(P=1 \mid C=1) = \frac{1}{2} = P(P=1)
$$

Finalmente, como $$(P(P \mid C) = P(P))$$ para **todas las combinaciones**, el cifrado **sí cumple con secreto perfecto** según la definición de Shannon.


**e)** What happens to perfect secrecy if we reuse the key for multiple bits?

Si reutilizamos la misma clave para cifrar **múltiples bits**, el cipher **ya no mantiene secreto perfecto**, ya que:

- Para un solo bit, cada ciphertext es independiente del plaintext gracias a la clave aleatoria de un solo bit.  
- Si usamos la misma clave \(K\) para varios bits, se introduce **dependencia entre los ciphertexts**.  
- Por ejemplo, si ciframos dos bits \(P_1, P_2\) con la misma clave \(K\):

$$
C_1 = P_1 \oplus K, \quad C_2 = P_2 \oplus K
$$

- Conociendo \(C1\) y \(C2\), un atacante puede calcular:

$$
C_1 \oplus C_2 = P_1 \oplus P_2
$$

- Esto **filtra información sobre los plaintexts** y permite inferencias, rompiendo la independencia requerida por Shannon.  

#### Por eso, reusar la clave compromete la seguridad y **el cipher deja de tener secreto perfecto**.



___
#### **Exercise 4: Entropy and Key Analysis**
Consider the following scenarios:

***Scenario A: A password system where:***
- Passwords are exactly 4 characters long
- Each character is chosen uniformly from {A, B, C, D}

***Scenario B: A Vigenère cipher with:***
- Key length = 3
- Each key character chosen uniformly from 26-letter alphabet

**a)** Calculate the entropy (in bits) for Scenario A

Datos:

- Cada contraseña tiene 4 caracteres.  
- Cada carácter se elige uniformemente de \(\{A, B, C, D\}\), es decir **4 opciones**.  

Sabemos que la fórmula de entropía para un espacio uniforme es:

$$
H = \log_2(N)
$$

donde \(N\) es el número de combinaciones posibles.  

Ahora, el número de combinaciones posibles:

$$
N = 4^4 = 256
$$

Por lo tanto, la entropía sería:

$$
H_A = \log_2(256) = 8 \text{ bits}
$$

In [20]:
import math

#Ponemos los datos
num_caracteres = 4 #longitud de la contraseña
opciones_por_caracter = 4 #A, B, C, D

#Número total de combinaciones
N = opciones_por_caracter ** num_caracteres

#Entropía en bits
H = math.log2(N)

print("Número total de combinaciones =", N)
print("Entropía =", H, "bits")


Número total de combinaciones = 256
Entropía = 8.0 bits


**b)** Calculate the entropy (in bits) for Scenario B

Sabemos que:

- Longitud de la clave: 3 caracteres  
- Cada carácter se elige uniformemente de 26 letras  

El número de combinaciones posibles:

$$
N = 26^3 = 17576
$$

Y por ende la entropía sería:

$$
H_B = \log_2(26^3) = 3 \cdot \log_2(26) \approx 14.09 \text{ bits}
$$

In [21]:
import math

#Ponemos los datos
long_clave = 3 # longitud de la clave
opciones_por_caracter = 26 # letras del alfabeto

#El número total de combinaciones
N = opciones_por_caracter ** long_clave

#Entropía en bits
H = long_clave * math.log2(opciones_por_caracter)

print("Número total de combinaciones=", N)
print("Entropía=", H, "bits")

Número total de combinaciones= 17576
Entropía= 14.101319154423278 bits


**c)** If an attacker can test 1000 keys per second, how long would it take to break each system in the worst case?

Tomemos en cuenta el **peor caso**, donde el atacante prueba todas las combinaciones posibles:

- En el escenario A:

$$
t_A = \frac{N_A}{1000} = \frac{256}{1000} \text{ s} \approx 0.256 \text{ s}
$$

- En el escenario B:

$$
t_B = \frac{N_B}{1000} = \frac{17576}{1000} \text{ s} \approx 17.576 \text{ s}
$$

**d)** Calculate the "unicity distance" concept: If English text has entropy ≈ 1.5 bits per character, how much ciphertext would theoretically be needed to uniquely determine a 3-character Vigenère key?

La definición de U, es la cantidad mínima de texto cifrado necesaria para determinar **únicamente** la clave.  

Fórmula aproximada:

$$
U \approx \frac{H(K)}{D}
$$

donde:

- \(H(K)\) = entropía de la clave  
- \(D\) = entropía por carácter del texto en bits  

Datos:

- \(H(K) = 14.09 \text{ bits}\) (del literal b)  
- Entropía del inglés ≈ 1.5 bits por carácter  

Por lo tanto:

$$
U \approx \frac{14.09}{1.5} \approx 9.39 \text{ caracteres}
$$

Entonces, se necesitan al menos **10 caracteres de ciphertext** para determinar la clave de 3 caracteres con alta probabilidad.